## Web Scrapping for Images

Demo code for automating the download of specific images giving a webpage

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import requests
import shutil
import chime
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from datetime import date, datetime
from IPython.core.debugger import Pdb #Pdb().set_trace()


# Define your login credentials
username = "xxxxxxxxxx@_____.com"
password = "················"

In [2]:
all_files = os.listdir('data/')
topics = [(theme
           .split('.')[0]
           .replace('C ', '')
           .replace('_', '')
           .replace(' 31','')
          ) for theme in all_files]
topics = dict(zip(range(len(topics)), topics))

#All files to be read
topics

{0: 'Technology',
 1: 'Social Awarenessupdated02112023-180454',
 2: 'Social Awareness',
 3: 'Finances and Businessupdated02112023-181812',
 4: 'Finances and Business',
 5: 'Technologyupdated02112023-170543',
 6: 'Luxury',
 7: 'Entertainment',
 8: 'Health and Wellbeing',
 9: 'Personal Aesthetics',
 10: 'Luxuryupdated02112023-185308',
 11: 'Tourism'}

In [3]:
update_file = int(input(f"which file do you want to update?: \n{topics}\n\n"))
print(f"\n\nThe file <<{all_files[update_file]}>> will be updated.")

which file do you want to update?: 
{0: 'Technology', 1: 'Social Awarenessupdated02112023-180454', 2: 'Social Awareness', 3: 'Finances and Businessupdated02112023-181812', 4: 'Finances and Business', 5: 'Technologyupdated02112023-170543', 6: 'Luxury', 7: 'Entertainment', 8: 'Health and Wellbeing', 9: 'Personal Aesthetics', 10: 'Luxuryupdated02112023-185308', 11: 'Tourism'}

11


The file <<C Tourism_.xlsx>> will be updated.


## Code to create all folders per topic

```Python
for topic in topics.values():
    path = f'Images/{topic}'
    try: 
        os.mkdir(path)
    except:
        pass
```

In [4]:
def read_not_published_data(file):
    df = pd.read_excel(f'data/{file}')
    not_published = df[df["published"]==0].reset_index(drop=True) #Not published yet
    return not_published


not_published = read_not_published_data( all_files[update_file] )

In [5]:
# Self note, make warning for false not being the only value
not_published["published"].unique()

array([False])

In [7]:
def get_img_url_from_Amplify(not_published):  
    not_published_update = not_published.copy()
    not_published_update["image_url"] = [""]*len(not_published_update)
    login_url = "https://xxxxxxxx.com/login"
    # Create a Chrome web driver
    driver = webdriver.Chrome()
    driver.implicitly_wait(60)
    # Open the login page
    driver.get(login_url)
    usr_box = driver.find_element("name", 'email')
    pas_box = driver.find_element("name", 'password')
    login_buttom = driver.find_element(By.CSS_SELECTOR, '.css-xxxxx')
    # Passing licence and getting to "Innovation"
    ActionChains(driver).move_to_element(usr_box).click(usr_box).send_keys(username).perform()
    ActionChains(driver).move_to_element(pas_box).click(pas_box).send_keys(password).perform()
    ActionChains(driver).move_to_element(login_buttom).click(login_buttom).perform()
    time.sleep(3) # Getting rid of tutorial placeholder
    ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    innov_buttom = driver.find_element(
        By.XPATH, '//a[contains(@class, "css-xxxxxx") and contains(text(), "xxxxxx")]'
    )
    ActionChains(driver).move_to_element(innov_buttom).click(innov_buttom).perform()
    for i, case_study_title in enumerate(not_published["Case Study Title"]):
        seach_box = driver.find_element(By.CSS_SELECTOR, '.css-xxxxxx')
        (# Clean search box
        ActionChains(driver).move_to_element(seach_box).click(seach_box)
            .key_down(Keys.CONTROL).send_keys('A').key_up(Keys.CONTROL)
            .send_keys(Keys.DELETE).perform()
        )
        (# Searching for specific case study in webpage
            ActionChains(driver).move_to_element(seach_box)
         .click(seach_box).send_keys(case_study_title).perform()
        )
        time.sleep(3)
        search_button = driver.find_element(By.CSS_SELECTOR, '.css-xxxxxx')
        ActionChains(driver).move_to_element(search_button).click(search_button).perform()
        try: # Created this block because there is high chances of failing in this stage
            time.sleep(3) 
            _element = driver.find_element(
                By.XPATH,'//img[@alt="image cover"]'
            )
        except:
            chime.warning()
            print('Send "c" when issue is fixed')
            Pdb().set_trace()
            _element = driver.find_element(
                By.XPATH,'//img[@alt="image cover"]'
            )
        ActionChains(driver).move_to_element(_element).click(_element).perform()
        time.sleep(3)        
        not_published_update.loc[i, "image_url"] = driver.find_element(
            By.XPATH,'//img[@alt="img cover"]'
        ).get_attribute('src')
        back_button = driver.find_element(By.CSS_SELECTOR, '.css-xxxxxx')
        ActionChains(driver).move_to_element(back_button).click(back_button).perform()
    return not_published_update

In [8]:
not_published_update = get_img_url_from_Amplify(not_published)
# For 116 elements: 16:15 -- 16:58

In [9]:
not_published_update.to_csv(
    "data/" + 
    f"{topics[update_file]}_updated_{datetime.now().strftime('%d%m%Y-%H%M%S')}" +
    ".csv",
    index=False
)

In [10]:
def save_images_from_updated_file(not_published_update):
    file = not_published_update.copy()
    path = f'Images/{topics[update_file]}'
    for i, case_study_title in enumerate(file["Case Study Title"]):
        image_url = file["image_url"][i]
        img_filename = f"{case_study_title.replace(' ', '_')[:25]}.{image_url.split('.')[-1]}"
        img_data = requests.get(image_url).content
        with open(path+"/"+img_filename, 'wb') as handler:
            handler.write(img_data)

In [11]:
save_images_from_updated_file(not_published_update)

In [12]:
chime.success()